In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file = pd.read_csv("data/suicide_rates.csv")
file2 = pd.read_csv("data/population/pop_totals.csv")

# preview file data using .head()
file.head()


,country,year,sex,age,suicides_no,population,suicides/100k pop,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
0,Albania,1987,male,15-24 years,21,312900,6.71,Albania1987,NaN,"2,156,624,900",796,Generation X
1,Albania,1987,male,35-54 years,16,308000,5.19,Albania1987,NaN,"2,156,624,900",796,Silent
2,Albania,1987,female,15-24 years,14,289700,4.83,Albania1987,NaN,"2,156,624,900",796,Generation X
3,Albania,1987,male,75+ years,1,21800,4.59,Albania1987,NaN,"2,156,624,900",796,G.I. Generation
4,Albania,1987,male,25-34 years,9,274300,3.28,Albania1987,NaN,"2,156,624,900",796,Boomers


In [2]:
file2.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54208.0,55434.0,56234.0,56699.0,57029.0,57357.0,...,102050.0,102565.0,103165.0,103776.0,104339.0,104865.0,105361.0,105846.0,106310.0,106766.0
1,Africa Eastern and Southern,AFE,"Population, total",SP.POP.TOTL,130836765.0,134159786.0,137614644.0,141202036.0,144920186.0,148769974.0,...,532760424.0,547482863.0,562601578.0,578075373.0,593871847.0,609978946.0,626392880.0,643090131.0,660046272.0,677243299.0
2,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996967.0,9169406.0,9351442.0,9543200.0,9744772.0,9956318.0,...,30117411.0,31161378.0,32269592.0,33370804.0,34413603.0,35383028.0,36296111.0,37171922.0,38041757.0,38928341.0
3,Africa Western and Central,AFW,"Population, total",SP.POP.TOTL,96396419.0,98407221.0,100506960.0,102691339.0,104953470.0,107289875.0,...,360285439.0,370243017.0,380437896.0,390882979.0,401586651.0,412551299.0,423769930.0,435229381.0,446911598.0,458803476.0
4,Angola,AGO,"Population, total",SP.POP.TOTL,5454938.0,5531451.0,5608499.0,5679409.0,5734995.0,5770573.0,...,24220660.0,25107925.0,26015786.0,26941773.0,27884380.0,28842482.0,29816769.0,30809787.0,31825299.0,32866268.0


In [3]:
countries = file['country'].unique()
#countries[100]


In [4]:
len(countries)

101

In [5]:
np.sort(file['year'].unique())

x_years = np.linspace(2006,2015,10)
x_years

array([2006., 2007., 2008., 2009., 2010., 2011., 2012., 2013., 2014.,
       2015.])

In [6]:
file['age'].unique()

array(['15-24 years', '35-54 years', '75+ years', '25-34 years',
       '55-74 years', '5-14 years'], dtype=object)

In [7]:
file['generation'].unique()

array(['Generation X', 'Silent', 'G.I. Generation', 'Boomers',
       'Millenials', 'Generation Z'], dtype=object)

In [8]:
max_suicide = file.groupby('suicides/100k pop').aggregate(np.max)
max_suicide

,country,year,sex,age,suicides_no,population,country-year,HDI for year,gdp_for_year ($),gdp_per_capita ($),generation
suicides/100k pop,,,,,,,,,,,
0.00,Uruguay,2016,male,75+ years,0,8673303,Uruguay2014,0.942,"997,007,926",126352,Silent
0.02,Thailand,2011,male,5-14 years,1,5821000,Thailand1994,0.651,"416,878,162,441",8906,Millenials
0.03,United Kingdom,2010,male,5-14 years,1,3778962,United Kingdom2010,0.906,"926,884,816,754",42486,Millenials
0.04,Spain,2015,male,5-14 years,2,5126281,Spain2015,0.874,"612,939,685,081",36869,Millenials
0.05,United Kingdom,2011,male,5-14 years,2,3949849,United Kingdom2003,0.870,"906,853,273,138",36289,Millenials
...,...,...,...,...,...,...,...,...,...,...,...
182.32,Republic of Korea,2010,male,75+ years,1152,631853,Republic of Korea2010,NaN,"1,094,499,338,703",22994,Silent
185.37,Republic of Korea,2011,male,75+ years,1276,688365,Republic of Korea2011,NaN,"1,202,463,682,634",25157,Silent
187.06,Suriname,2012,male,75+ years,10,5346,Suriname2012,0.711,"4,980,000,000",10227,Silent


In [9]:
suicide_rates = file.drop(labels = 'gdp_per_capita ($)', axis = 1)
suicide_rates = suicide_rates.drop(labels = 'country-year', axis = 1)
suicide_rates = suicide_rates.drop(labels = 'HDI for year', axis = 1)
suicide_rates = suicide_rates.drop(labels = 'age', axis =1)
suicide_rates = suicide_rates.drop(labels = 'sex', axis =1)
suicide_rates = suicide_rates.drop(labels = 'population', axis =1)

In [10]:
suicide_rates

,country,year,suicides_no,suicides/100k pop,gdp_for_year ($),generation
0,Albania,1987,21,6.71,"2,156,624,900",Generation X
1,Albania,1987,16,5.19,"2,156,624,900",Silent
2,Albania,1987,14,4.83,"2,156,624,900",Generation X
3,Albania,1987,1,4.59,"2,156,624,900",G.I. Generation
4,Albania,1987,9,3.28,"2,156,624,900",Boomers
...,...,...,...,...,...,...
27815,Uzbekistan,2014,107,2.96,"63,067,077,179",Generation X
27816,Uzbekistan,2014,9,2.58,"63,067,077,179",Silent
27817,Uzbekistan,2014,60,2.17,"63,067,077,179",Generation Z
27818,Uzbekistan,2014,44,1.67,"63,067,077,179",Generation Z


In [11]:
# for index, row in suicide_rates.iterrows(): 
#     print("Total income in "+ row["Date"]+ " is:"+str(row["Income_1"]+row["Income_2"]))

In [12]:
suicide_rates.groupby("year").aggregate(np.mean)

,suicides_no,suicides/100k pop
year,,
1985,201.498264,11.826198
1986,209.496528,11.423333
1987,195.743827,11.644213
1988,205.826531,12.709405
1989,256.801282,12.879071
1990,251.772135,12.862956
1991,257.838542,13.438880
1992,271.119231,13.498564
1993,284.057692,13.833705


In [13]:
#suicide_10= suicide_rates[suicide_rates['year'] >= 2006] and suicide_rates[suicide_rates['year'] <=2016]
suicide_10=suicide_rates[(suicide_rates['year'] >= 2006) & (suicide_rates['year'] <= 2015)]
suicide_10

,country,year,suicides_no,suicides/100k pop,gdp_for_year ($),generation
204,Albania,2006,0,0.00,"8,992,642,349",Millenials
205,Albania,2006,0,0.00,"8,992,642,349",Generation X
206,Albania,2006,0,0.00,"8,992,642,349",Boomers
207,Albania,2006,0,0.00,"8,992,642,349",Millenials
208,Albania,2006,0,0.00,"8,992,642,349",Silent
...,...,...,...,...,...,...
27815,Uzbekistan,2014,107,2.96,"63,067,077,179",Generation X
27816,Uzbekistan,2014,9,2.58,"63,067,077,179",Silent
27817,Uzbekistan,2014,60,2.17,"63,067,077,179",Generation Z
27818,Uzbekistan,2014,44,1.67,"63,067,077,179",Generation Z


I want to combine the rows that have the same country and year, 
not worrying about the generation for now, to add up the total suicide count for each country for
each year. Then find the average suicide count for the decade. Then I'll take the 3 countries
with the least amount of suicides (not counting the ones that have zeroes for every
single generation). I'll look at the top country in tht list and break it back down into 
generation to find he happiest one. If time permits, I will also find the saddest generation.


In [14]:
z = suicide_10.groupby(["country", "year"]).aggregate(np.sum)
z

suicides_no  suicides/100k pop
country    year                                
Albania    2006            0               0.00
           2007          124              65.85
           2008          160              71.05
           2009            0               0.00
           2010           96              41.66
...                      ...                ...
Uzbekistan 2010         1464              63.23
           2011         1640              67.14
           2012         1835              77.53
           2013         1950              78.86
           2014         2095              85.12

[820 rows x 2 columns]

In [15]:
# z = suicide_10.groupby(["country"]).aggregate(np.sum)
# z

In [16]:
z.reset_index(0).reset_index(drop = False)
z.head(20)

suicides_no  suicides/100k pop
country             year                                
Albania             2006            0               0.00
                    2007          124              65.85
                    2008          160              71.05
                    2009            0               0.00
                    2010           96              41.66
Antigua and Barbuda 2006            0               0.00
                    2007            1               7.98
                    2008            0               0.00
                    2009            0               0.00
                    2012            0               0.00
                    2013            0               0.00
                    2014            0               0.00
                    2015            1              15.62
Argentina           2006         3108             128.22
                    2007         2946             118.35
                    2008         3059             118.11
                    2009         2884             107.32
                    2010         2943             111.24
                    2011         2912             105.87
                    2012         3248             119.82

In [17]:
#z.get_level_values("country")
#z.xs("country")#["country"]["year"]
#z.loc["Albania"]
z.loc[("Albania",2006)]


suicides_no          0.0
suicides/100k pop    0.0
Name: (Albania, 2006), dtype: float64

In [18]:
#how to count rows in a multi index
u = z.reset_index().groupby('country')['year'].nunique()
u = pd.DataFrame(u)
u

,year
country,
Albania,5
Antigua and Barbuda,8
Argentina,10
Armenia,10
Aruba,6
...,...
United Arab Emirates,5
United Kingdom,10
United States,10


In [19]:
z["years_of_data"] = u["year"]
z.head()

suicides_no  suicides/100k pop  years_of_data
country year                                               
Albania 2006            0               0.00            NaN
        2007          124              65.85            NaN
        2008          160              71.05            NaN
        2009            0               0.00            NaN
        2010           96              41.66            NaN

In [20]:
len(u)

95

In [21]:
len(u[u == 5])

95

In [26]:
#average suicide count from 2006-2015 per country
a = z.groupby("country").agg({"suicides_no":lambda x:x.sum(), "suicides/100k pop":lambda x:x.sum()})
a["years_of_data"] = u["year"]
a

,suicides_no,suicides/100k pop,years_of_data
country,,,
Albania,380,178.56,5
Antigua and Barbuda,2,23.60,8
Argentina,30391,1135.83,10
Armenia,683,381.27,10
Aruba,41,424.24,6
...,...,...,...
United Arab Emirates,526,78.84,5
United Kingdom,44071,802.56,10
United States,387385,1601.48,10


In [27]:
a["avg_suicides"] = a["suicides_no"] / a['years_of_data']
a

,suicides_no,suicides/100k pop,years_of_data,avg_suicides
country,,,,
Albania,380,178.56,5,76.000000
Antigua and Barbuda,2,23.60,8,0.250000
Argentina,30391,1135.83,10,3039.100000
Armenia,683,381.27,10,68.300000
Aruba,41,424.24,6,6.833333
...,...,...,...,...
United Arab Emirates,526,78.84,5,105.200000
United Kingdom,44071,802.56,10,4407.100000
United States,387385,1601.48,10,38738.500000


In [42]:
b = a.sort_values(by=["avg_suicides"])
b.head(10)

,suicides_no,suicides/100k pop,years_of_data,avg_suicides
country,,,,
Antigua and Barbuda,2,23.60,8,0.250000
Grenada,7,96.97,10,0.700000
Maldives,8,21.74,4,2.000000
Barbados,18,84.11,8,2.250000
Saint Vincent and Grenadines,47,583.89,10,4.700000
Bahamas,41,144.59,8,5.125000
Seychelles,56,899.08,10,5.600000
Aruba,41,424.24,6,6.833333
Saint Lucia,65,474.47,8,8.125000


In [28]:
# b = z.groupby("country").agg({"suicides_no":lambda x:x.sum() / , "suicides/100k pop":lambda x:x.sum() / 10})

In [ ]:
#printing generational suicide data for the past 10 years

generational_suicide_10 = suicide_10.groupby("generation").sum("suicides/100k pop")
generational_suicide_10 = generational_suicide_10.drop(labels = 'year', axis =1)
generational_suicide_10 = generational_suicide_10.drop(labels = 'suicides/100k pop', axis =1)
generational_suicide_10

In [ ]:
global_suicide_10 = suicide_10.groupby("year").sum()
global_suicide_10

In [ ]:
y_sui_total = global_suicide_10["suicides/100k pop"]
y_sui_totals = np.array(y_sui_total)
y1 = list(y_sui_totals)


In [ ]:
#file2['Year']
world = file2.loc[259][-15:-5]
world_pop = world.transpose()
y2 = list(world_pop)

In [ ]:
percent_suicide = []
for i in y1:
    for j in y2:
        if y1.index(i) == y2.index(j):
            new_percent = i/j * 100
            percent_suicide.append(new_percent)
percent_suicide

In [ ]:
avg_glb_percent = np.mean(percent_suicide)
avg_glb_percent

In [ ]:
data = []
for i in y1:
    for j in y2:
        for k in percent_suicide:
            if y1.index(i) == y2.index(j) == percent_suicide.index(k):
                data.append([i,j,k])
data

In [ ]:
glb_sui = pd.DataFrame(data, index = x_years, columns = ['No. of Suicides', 'Global Population', 'Percentage of the Population'])
glb_sui

In [ ]:
#Bar graph depicting the number of suicides per year from 2006-2015
ax, fig = plt.subplots(sharex=True,sharey=True)


plt.title("Global Population in the Past Decade")
plt.xlabel("Year")
plt.ylabel("No. of People (in billions)")
plt.xticks(range(2006,2016,1))
#plt.yticks(range(0,700000000,1000000))
#x-axis values
x_years = np.linspace(2006,2015,10)

#y-axis values
y1 = list(world_pop)
#y2 = list(world_pop)


plt.plot(y1, color = 'brown')
#plt.plot(y2, alpha = .7, color = 'brown')

plt.tight_layout()




#plt.subplot(1,2,1, facecolor='red')
# ax, fig = plt.subplots(1,1,sharex=True,sharey=True)


#plt.bar(years, A, color = 'b', label='A')
#plt.bar(years, increment, color = 'r', bottom = A, label='increment')

#plt.legend()



# ax, fig = plt.subplots(1,1,sharex=True,sharey=True)
# plt.bar(x_years, list(world_pop))
plt.savefig("images/Global_pop_last_decade")

In [ ]:
plt.title("Number of Suicides in the Past Decade")
plt.xlabel("Year")
plt.ylabel("No. of People")
plt.xticks(range(2006,2016,1))
#plt.yticks(range(0,700000000,1000000))
#x-axis values
x_years = np.linspace(2006,2015,10)

#y-axis values

y2 = list(y_sui_totals)


#plt.plot(y1, color = 'red')
plt.plot(y2, alpha = .7, color = 'red')

plt.tight_layout()
plt.savefig("images/Suicide_count_last_decade")

In [ ]:
#percent of population has committed suicide
ax, fig = plt.subplots()

plt.bar(x_years, percent_suicide, color = '#7fcdbb', alpha = 0.8)
plt.xticks(range(2006,2016,1))
#plt.yticks(range(0,1,10))
plt.yticks(np.arange(0, max(percent_suicide),5))
#plt.set_size_inches(2,4)
plt.tight_layout()

In [ ]:
global_avg = global_suicide_10['suicides_no'].mean()
global_avg

In [ ]:
#y1 = suicide totals for the decade, y2 = global population for the the decade
y1, y2

In [ ]:
suicide_10.groupby("country").sum()

In [ ]:
by_year=suicide_10.groupby('year').aggregate(np.max)
by_year

In [ ]:
by_country=by_year.groupby('country').aggregate(np.max)
by_country.head()

In [ ]:
#by_country.groupby("year").sum()

In [ ]:
df.loc[df['team'] == 'A', 'points'].sum()

In [ ]:
suicide_10.groupby("country").sum()